In [1]:
import json
import os
import shutil
from tqdm import tqdm
from PIL import Image
import numpy as np


In [2]:
def load_json_data(data_path):
    with open(data_path, 'r') as f:
        data = json.load(f)
    return data

def save_json_data(data, data_path):
    with open(data_path, 'w') as f:
        json.dump(data, f) 

In [3]:
# exemplar_dict_path = '../datasets/metadata/coco_exemplar_dict.json'
exemplar_dict_path = '../datasets/metadata/pascal_voc_exemplar_dict.json'
exemplar_dict = load_json_data(exemplar_dict_path)

In [4]:
len(exemplar_dict)

20

### 使用实例数选择图片
因为一张图片上可能有多个实例

In [6]:
def get_crop(img, bb, context=0.0, square=True):
    # print(bb)
    x1, y1, w, h = bb
    W, H = img.size
    y, x = y1 + h / 2.0, x1 + w / 2.0
    h, w = h * (1. + context), w * (1. + context)
    if square:
        w = max(w, h)
        h = max(w, h)
    # print(x, y, w, h)
    x1, x2 = x - w / 2.0, x + w / 2.0
    y1, y2 = y - h / 2.0, y + h / 2.0
    # print([x1, y1, x2, y2])
    x1, x2 = max(0, x1), min(W, x2)
    y1, y2 = max(0, y1), min(H, y2)
    # print([x1, y1, x2, y2])
    bb_new = [int(c) for c in [x1, y1, x2, y2]]
    # print(bb_new)
    crop = img.crop(bb_new)
    return crop


def run_crop(d, source_path, context=0.4, square=True):
    dataset = d['dataset']
    # with open(file_name, "rb") as f:
    img = Image.open(source_path)
    if dataset == "imagenet21k":
        bb = [0, 0, 0, 0]
        return img
    elif dataset == "lvis":
        bb = [
            int(c)
            for c in [
                d['bbox'][0] // 1,
                d['bbox'][1] // 1,
                d['bbox'][2] // 1 + 1,
                d['bbox'][3] // 1 + 1
            ]
        ]
    elif dataset == "visual_genome":
        bb = [int(c) for c in [d['x'], d['y'], d['w'], d['h']]]
    return get_crop(img, bb, context=context, square=square)


In [8]:
# lvis使用
def copy_files_by_instances_num_lvis(data_dict, filter, max_nums, context=0.4, square=True):
    total_count = 0
    for key, items in tqdm(data_dict.items(), desc="Copying files", unit="category"):
        for item in items[:max_nums]:
        # for item in items:
            total_count = total_count + 1
            
            dataset = item['dataset']
            category_id = item['category_id']
            file_name = item['file_name']
            if dataset == 'imagenet21k':
                folder_prefix = '/imagenet/imagenet21k_P/'
                dataset_abbr = 'in21k'
                object_id = ''
            elif dataset == 'lvis':
                folder_prefix =  '/coco/'
                dataset_abbr = 'lvis'
                object_id = item['id']
            elif dataset == 'visual_genome':
                folder_prefix =  '/VisualGenome/'
                dataset_abbr = 'vg'
                object_id = item['object_id']

            source_path = f"/raid/mqsen/mm-ovod/datasets/{folder_prefix}/{file_name}"
            
            folder_path = f"../exemplar_prototype/lvis/{filter}/regions/{category_id}"
            
            
            # 检查目标文件夹是否存在，不存在则创建
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            destination_path = os.path.join(folder_path,f'{category_id}_{dataset_abbr}_{object_id}_{os.path.basename(source_path)}')
            
            exemplar_region = run_crop(item, source_path, context, square)
            exemplar_region.save(destination_path)
            
            # # 检查目标文件夹是否存在，不存在则创建
            # if not os.path.exists(folder_path):
            #     os.makedirs(folder_path)
                
            # if dataset_abbr == 'lvis':
                
            # else:
            #     # 构建目标文件路径
            #     destination_path = os.path.join(folder_path,f'{category_id}_{dataset_abbr}_{os.path.basename(source_path)}')
            #     # 复制文件
            #     shutil.copy(source_path, destination_path)
            # # print(f"Copied {source_path} to {destination_path}")
    print("total count: ", total_count)


In [6]:
# coco使用
def copy_files_by_instances_num_coco(data_dict, filter, max_nums, context=0.4, square=True):
    total_count = 0
    for cat_info in tqdm(data_dict, desc="Copying files", unit="category"):
        for item in cat_info["exemplars"][:max_nums]:
            total_count = total_count + 1
            
            dataset = item['dataset']
            category_id = cat_info['id']
            file_name = item['file_name']
            if dataset == 'imagenet21k':
                folder_prefix = '/imagenet/imagenet21k_P/'
                dataset_abbr = 'in21k'
                object_id = ''
            elif dataset == 'lvis':
                folder_prefix =  '/coco/'
                dataset_abbr = 'lvis'
                object_id = item['id']
            elif dataset == 'visual_genome':
                folder_prefix =  '/VisualGenome/'
                dataset_abbr = 'vg'
                object_id = item['object_id']

            source_path = f"/raid/mqsen/mm-ovod/datasets/{folder_prefix}/{file_name}"
            
            folder_path = f"../exemplar_prototype/coco/{filter}/regions/{category_id}"
            
            
            # 检查目标文件夹是否存在，不存在则创建
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            destination_path = os.path.join(folder_path,f'{category_id}_{dataset_abbr}_{object_id}_{os.path.basename(source_path)}')
            
            exemplar_region = run_crop(item, source_path, context, square)
            exemplar_region.save(destination_path)
            
            # # 检查目标文件夹是否存在，不存在则创建
            # if not os.path.exists(folder_path):
            #     os.makedirs(folder_path)
                
            # if dataset_abbr == 'lvis':
                
            # else:
            #     # 构建目标文件路径
            #     destination_path = os.path.join(folder_path,f'{category_id}_{dataset_abbr}_{os.path.basename(source_path)}')
            #     # 复制文件
            #     shutil.copy(source_path, destination_path)
            # # print(f"Copied {source_path} to {destination_path}")
    print("total count: ", total_count)

In [7]:
# pascal_voc使用
def copy_files_by_instances_num_pascal(data_dict, filter, max_nums, context=0.4, square=True):
    total_count = 0
    for cat_info in tqdm(data_dict, desc="Copying files", unit="category"):
        for item in cat_info["exemplars"][:max_nums]:
            total_count = total_count + 1
            
            dataset = item['dataset']
            category_id = cat_info['id']
            file_name = item['file_name']
            if dataset == 'imagenet21k':
                folder_prefix = '/imagenet/imagenet21k_P/'
                dataset_abbr = 'in21k'
                object_id = ''
            elif dataset == 'lvis':
                folder_prefix =  '/coco/'
                dataset_abbr = 'lvis'
                object_id = item['id']
            elif dataset == 'visual_genome':
                folder_prefix =  '/VisualGenome/'
                dataset_abbr = 'vg'
                object_id = item['object_id']

            source_path = f"/raid/mqsen/mm-ovod/datasets/{folder_prefix}/{file_name}"
            
            folder_path = f"../exemplar_prototype/pascal_voc/{filter}/regions/{category_id}"
            
            
            # 检查目标文件夹是否存在，不存在则创建
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            destination_path = os.path.join(folder_path,f'{category_id}_{dataset_abbr}_{object_id}_{os.path.basename(source_path)}')
            
            exemplar_region = run_crop(item, source_path, context, square)
            exemplar_region.save(destination_path)
            
            # # 检查目标文件夹是否存在，不存在则创建
            # if not os.path.exists(folder_path):
            #     os.makedirs(folder_path)
                
            # if dataset_abbr == 'lvis':
                
            # else:
            #     # 构建目标文件路径
            #     destination_path = os.path.join(folder_path,f'{category_id}_{dataset_abbr}_{os.path.basename(source_path)}')
            #     # 复制文件
            #     shutil.copy(source_path, destination_path)
            # # print(f"Copied {source_path} to {destination_path}")
    print("total count: ", total_count)

In [8]:
maxnum = 10
context = 0.4
square = False
filter = f'ins{maxnum}_context{context}_square{square}'
copy_files_by_instances_num_pascal(exemplar_dict, filter, maxnum, context, square)
 

Copying files:   0%|          | 0/20 [00:00<?, ?category/s]


FileNotFoundError: [Errno 2] No such file or directory: '/raid/mqsen/mm-ovod/datasets//imagenet/imagenet21k_P//train/n02691156/n02691156_39394.JPEG'

### lvis clip debug

In [ ]:
def get_crop(img, bb, context=0.0, square=True):
    # print(bb)
    x1, y1, w, h = bb
    W, H = img.size
    y, x = y1 + h / 2.0, x1 + w / 2.0
    h, w = h * (1. + context), w * (1. + context)
    if square:
        w = max(w, h)
        h = max(w, h)
    # print(x, y, w, h)
    x1, x2 = x - w / 2.0, x + w / 2.0
    y1, y2 = y - h / 2.0, y + h / 2.0
    # print([x1, y1, x2, y2])
    x1, x2 = max(0, x1), min(W, x2)
    y1, y2 = max(0, y1), min(H, y2)
    # print([x1, y1, x2, y2])
    bb_new = [int(c) for c in [x1, y1, x2, y2]]
    # print(bb_new)
    crop = img.crop(bb_new)
    return crop


def run_crop(d, source_path, context=0.4, square=True):
    dataset = d['dataset']
    # with open(file_name, "rb") as f:
    img = Image.open(source_path)
    if dataset == "imagenet21k":
        bb = [0, 0, 0, 0]
        return img
    elif dataset == "lvis":
        bb = [
            int(c)
            for c in [
                d['bbox'][0] // 1,
                d['bbox'][1] // 1,
                d['bbox'][2] // 1 + 1,
                d['bbox'][3] // 1 + 1
            ]
        ]
    elif dataset == "visual_genome":
        bb = [int(c) for c in [d['x'], d['y'], d['w'], d['h']]]
    return get_crop(img, bb, context=context, square=square)